# This script processes the optim 4 (minRadius - maxNumber) results of the minRadius and maxNumber studies

1. Compile o3memo (includes minRadius=0.1 by default) and o4maxnum (has maxNumber=32 by default) :
```
make bin/o3memo
make bin/o4maxnum
```
2. Run the benchmarking script
```
cd scripts
python3 run_minrad_level.py
python3 run_maxnum_level.py
```
3. The output that we will process is `o4_minradius_<hostname>.csv` and `o4_maxnumber_<hostname>.csv`, that should be already saved in the `Results` folder.



In [ ]:
import os
# get the hostname of the server
hostname = os.popen("hostname").read().strip()
# ensure the directory exists
os.makedirs(hostname, exist_ok=True)
# ensure the files exist
file_list = [                   
                f'o4_minradius_{hostname}.csv',
                f'o4_maxnumber_{hostname}.csv',
            ]

for o4_file in file_list:
    # if it is not in Results
    if not os.path.exists(o4_file):
        # is the file already in the directory?
        assert os.path.exists(os.path.join(hostname, o4_file)), f'File {o4_file} not found: something went wrong with the baseline benchmark.'
    # if it is in Results
    else:
        # copy the file to the directory
        assert os.system(f'mv {o4_file} {hostname}/') == 0, f'Failed to move {o4_file} to {hostname}/'
# add the path to all the names in the list
for i in range(len(file_list)):
    file_list[i] = os.path.join(hostname, file_list[i])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import sys
from statistics import mean
from common.utils import get_nprocs, get_best_comb_3d, get_nested_values

# number of threads used in the execution
num_threads = get_nprocs()
# max num threads
maxth = max(num_threads) # num_threads[-1]

def tokenize(filename, type='minRadius'):
    results ={}

    with open(filename) as f:
        next(f) # skip header
        for line in f:
            # token format: ../bin/o4maxnum ../bin/data/AlcoyH.xyz 8 0.1 8 3 0.628122 0.346899 99.6489
            # token position: ExecName(0) InputFile(1) NumThreads(2) MinRadius(3) MaxNumber(4) Level(5) TreeTime(6) OwmTime(7) Correctness(8)
            tokens = line.split()
            name=tokens[1].split("/")[3][:-5]
            level=int(tokens[5])
            nth = int(tokens[2])
            if type == 'minRadius':
                minRad = float(tokens[3])
            else:
                minRad = int(tokens[4])
            if name not in results:
                results[name]={}
            if level not in results[name]:
                results[name][level]={}
            if minRad not in results[name][level]:
                results[name][level][minRad]={}
            results[name][level][minRad][nth]=[float(tokens[6])] # tree construction time
            results[name][level][minRad][nth].append(float(tokens[7])) # owm time
            results[name][level][minRad][nth].append(float(tokens[6])+float(tokens[7])) # total time
            results[name][level][minRad][nth].append(float(tokens[8])) # accuracy
    return results

resR = tokenize(os.path.join(hostname, f'o4_minradius_{hostname}.csv'), "minRadius")
resN = tokenize(os.path.join(hostname, f'o4_maxnumber_{hostname}.csv'), "maxNumber")

# Tree construction execution time for each cloud depending on level and minRadius

In [ ]:
def plot_alltimes(results,phase,ncurves,maxNummode=False):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-','.-','<-','>-','1-','2-','3','4','8','p','h','H','*','D','d','|','_']

    titles=['Tree construction time','OWM time','Total time']
    # get the cloud list
    cloud_list = list(results.keys())
    # get the different levels evaluated
    levels = list(results[cloud_list[0]].keys())
    # get the different minrad/maxnum values evaluated
    # if maxNummode:
    #     labels=list(results[cloud_list[0]][levels[0]].keys())[1:-4:step]
    # else:
    #     labels=list(results[cloud_list[0]][levels[0]].keys())[::step]
    labels=list(results[cloud_list[0]][levels[0]].keys())
    # get the number of threads
    vnth = list(results[cloud_list[0]][levels[0]][labels[0]].keys())
    # define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    # iterate through the clouds
    for i,name in enumerate(cloud_list):
        # # plot all the conf curves for each cloud
        # for minrad,m in zip(labels,marks):
        #     #print(name," ",[results[name][i][minrad][phase] for i in x])
        #     axs[i].plot(levels, np.array([results[name][i][minrad][vnth[-1]][phase] for i in levels]),m, linewidth=linew, markersize=markers, label=f'{minrad}')

        # plot the best configuration
        bestconf = get_best_comb_3d(results[name], 2)
        bestlevel = bestconf[0]
        bestminrad = bestconf[1]
        bestnth = bestconf[2]
        axs[i].plot(levels, np.array([results[name][i][bestminrad][bestnth][phase] for i in levels]), 'H--', linewidth=linew, markersize=markers, label=f'{bestminrad} BEST')

        # get 4 minrad values around the best configuration
        minrad_list = list(results[name][bestlevel].keys())
        minrad_list = [x for x in minrad_list if x != bestminrad]
        minrad_list = sorted(minrad_list, key=lambda x: abs(x-bestminrad))
        minrad_list = minrad_list[:ncurves]

        # plot all the conf curves for each cloud
        for minrad,m in zip(minrad_list,marks):
            axs[i].plot(levels, np.array([results[name][i][minrad][vnth[-1]][phase] for i in levels]),m, linewidth=linew, markersize=markers, label=f'{minrad}')
        
        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Level', fontsize=xlabelfs)
        axs[i].set_xticks(levels)
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        axs[i].legend(loc='best', fontsize= 14)
        axs[i].grid()

    if maxNummode:
        fig.suptitle(f'O4 MaxNum mode {titles[phase]} (sec.) @ {hostname}', fontweight='bold', fontsize=18)
    else:
        fig.suptitle(f'O4 MinRad mode {titles[phase]} (sec.) @ {hostname}', fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    #axs[i].show()
    if(phase==0):
        if(maxNummode):
            plt.savefig(f"{hostname}/MaxNum-Level-{hostname}.pdf")
        else:
            plt.savefig(f"{hostname}/MinRad-Level-{hostname}.pdf")

# plot_alltimes(3,resR,0)
# plot_alltimes(3,resR,1)
plot_alltimes(resR,2,6)

In [ ]:
#For MaxNumber
# plot_alltimes(2,resN,0,True)
# plot_alltimes(2,resN,1,True)
plot_alltimes(resN,2,6,True)

In [ ]:
# compute the efficiency loss for a configuration with respect to the best configuration
def efficiency_loss(results, phase):
    # get the cloud list
    cloud_list = list(results.keys())
    # get the different levels evaluated
    levels = list(results[cloud_list[0]].keys())
    # get the different minrad values evaluated
    labels=list(results[cloud_list[0]][levels[0]].keys())

    # compute the efficiency loss for each cloud
    eff_loss = {}
    for name in cloud_list:
        eff_loss[name] = {}
        # get the best configuration
        bestconf = get_best_comb_3d(results[name], 2)
        bestlevel = bestconf[0]
        bestminrad = bestconf[1]
        bestnth = bestconf[2]
        besttime = results[name][bestlevel][bestminrad][bestnth][phase]
        for level in levels:
            eff_loss[name][level] = {}
            for minrad in labels:
                eff_loss[name][level][minrad] = besttime / results[name][level][minrad][bestnth][phase]
    return eff_loss

minradius = 0.3
eff_loss = efficiency_loss(resR, 2)
# print the loss for each cloud at an specific minradius
for cloud in resR:
    print(f'Efficiency loss for {cloud}')
    # print only the best case
    print(max([eff_loss[cloud][x][minradius] for x in eff_loss[cloud]]))


In [ ]:

def best_minradius(resR, eff_loss):
    # instead manually selecting the minradius, we can get the minradius that gives the minimum efficiency loss
    best_minrad = {}
    # get the cloud list
    cloud_list = list(resR.keys())
    # get the different levels evaluated
    levels = list(resR[cloud_list[0]].keys())
    # get the different minrad values evaluated
    labels=list(resR[cloud_list[0]][levels[0]].keys())

    for cloud in resR:
        for level in levels:
            if level not in best_minrad:
                best_minrad[level] = {}
            for minrad in labels:
                # if the minrad is not in the best_minrad dictionary
                if minrad not in best_minrad[level]:
                    best_minrad[level][minrad] = eff_loss[cloud][level][minrad]
                else:
                    best_minrad[level][minrad] += eff_loss[cloud][level][minrad]
    
    # divide by the number of clouds
    for level in levels:
        for minrad in labels:
            best_minrad[level][minrad] /= len(cloud_list)

    return best_minrad

best_minrad = best_minradius(resR, eff_loss)

print(f'Best minradius: {best_minrad}')

# choose the best level and minRadius
flatten = [(level, minrad, 1-best_minrad[level][minrad]) for level in best_minrad for minrad in best_minrad[level]]
best_conf = min(flatten, key=lambda x: x[2])
print(f'Best configuration: lelvel={best_conf[0]}, minRadius={best_conf[1]}, loss={best_conf[2]}')


In [ ]:
# plot the best minradius curve
def plot_best_minradius(best_minrad):
    # plot the best minradius curve
    plt.figure(figsize=(8, 6))
    # get the list of levels
    levels = list(best_minrad.keys())
    # get the list of minrad values
    minrad = list(best_minrad[levels[0]].keys())
    # plot the level curves
    for level in levels:
        plt.plot(minrad, [1-best_minrad[level][x] for x in minrad], 'o-', label=f'Level {level}')
    plt.xlabel('MinRadius', fontsize=16)
    plt.xticks(minrad)
    plt.ylabel('Efficiency loss', fontsize=16)
    plt.title(f'Best minradius curve @ {hostname}', fontweight='bold', fontsize=18)
    plt.legend(loc='best', fontsize=14)
    plt.grid()

plot_best_minradius(best_minrad)



# Figures to compare different number of threads

In [ ]:
def compare_nth(results,step=1,phase=2,maxNummode=False):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-','.-','<-','>-','1-','2-','3','4','8','p','h','H','*','D','d','|','_']

    titles=['Tree construction time','OWM time','Total time']
    # get the cloud list
    cloud_list = list(results.keys())
    # get the different levels evaluated
    levels = list(results[cloud_list[0]].keys())
    # get the labels, a.k.a. the minrad or maxnum
    curve_label = ''
    if maxNummode:
        labels=list(results[cloud_list[0]][levels[0]].keys())[1:-4:step]
        curve_label = 'mN'
    else:
        labels=list(results[cloud_list[0]][levels[0]].keys())[::step]
        curve_label = 'mR'
    # get the number of threads
    vnth = list(results[cloud_list[0]][levels[0]][labels[0]].keys())
    # define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    # iterate through the clouds
    for i,name in enumerate(cloud_list):
        # iterate through the number of threads
        for nth,m in zip(vnth,marks):
            # plot the best configuration
            _,bestconf,_,_ = get_best_comb_3d(results[name], phase)
            # bestconf will be either the best minrad or the best maxnum
            axs[i].plot(levels, np.array([results[name][i][bestconf][nth][phase] for i in levels]), m, linewidth=linew, markersize=markers, label=f'{nth} th')
        
        axs[i].set_title(f'{name} {curve_label}: {bestconf}',fontsize=16)
        axs[i].set_xlabel('Level', fontsize=xlabelfs)
        axs[i].set_xticks(levels)
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()

    if maxNummode:
        fig.suptitle(f'{titles[phase]} O4 MaxNumber (sec.) @ {hostname.upper()}', fontweight='bold', fontsize=18)
    else:
        fig.suptitle(f'{titles[phase]} O4 MinRadius (sec.) @ {hostname.upper()}', fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(loc='best', fontsize= 14)
    #axs[i].show()
    # if(phase==0):
    #     if(maxNummode):
    #         plt.savefig(f"{hostname}/MaxNum-Level-{hostname}.pdf")
    #     else:
    #         plt.savefig(f"{hostname}/MinRad-Level-{hostname}.pdf")

compare_nth(resR, step=3, phase=2)

In [ ]:
compare_nth(resN, step=2, phase=2, maxNummode=True)

In [ ]:
import pandas as pd

def plot_improvement(res):

    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-']

    df=pd.read_csv(os.path.join(hostname, f'All_Optimizations-{hostname}.csv'),sep=';')
    # keep only baseline
    df = df[df['Optimization']=='Baseline'].copy()
    # initialize the columns
    df.insert(4,"Total",0)
    df['Total']=df['TimeTree']+df['TimeOWM']

    # get the cloud list
    cloud_list = list(res.keys())

    # add the results to the dataframe
    for nth in get_nested_values(res, 3):
        for name in res:
            # get the best configuration for a fixed number of threads
            flatten = [(x,y,res[name][x][y][nth][2]) for x in res[name] for y in res[name][x]]
            bestlevel,bestlabel,_ = min(flatten, key=lambda item:item[2])
            # add a new line to the dataframe
            df.loc[len(df)] = [
                f'o4@{nth}',
                name, 
                res[name][bestlevel][bestlabel][nth][0], 
                res[name][bestlevel][bestlabel][nth][1], 
                res[name][bestlevel][bestlabel][nth][2],
                bestlevel,
                bestlabel,
                ]
            
    df['TreeSp']=1.
    df['OWMSp']=1.
    df['TotalSp']=1.
    #Speedup of one optimization w.r.t. baseline
    for i in range(4,len(df),4):
        for j in range(4):
            df.loc[i+j,'TreeSp']=df['TimeTree'][j]/df['TimeTree'][i+j]
            df.loc[i+j,'OWMSp']=df['TimeOWM'][j]/df['TimeOWM'][i+j]
            df.loc[i+j,'TotalSp']=df['Total'][j]/df['Total'][i+j]    

    # display(df)

    # drop baseline
    all_df = df[~df['Optimization'].str.contains('Baseline')].copy()
    # get an array
    all_a = np.array(all_df.loc[:,'TreeSp':'TotalSp'])
    # get the labels from the dataframe
    xlab = all_df['Optimization'].unique()
    # drop from each label the 'o4@' prefix
    xlab = [x.split('@')[-1] for x in xlab]
    # get the results by cloud
    bycloud={}
    for i,name in enumerate(cloud_list):
        bycloud[name]=all_a[i::4].T

    #fig = plt.figure()
    labels=['OWM Trav.','Tree Const.','Total']
    x=np.arange(1,len(xlab)+1)
    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True, sharey=True)
    for i,name in enumerate(cloud_list):
        axs[i].plot(x, bycloud[name][1], marks[0], linewidth=linew, markersize=markers)
        axs[i].plot(x, bycloud[name][0], marks[1], linewidth=linew, markersize=markers)
        axs[i].plot(x, bycloud[name][2], marks[2], linewidth=linew, markersize=markers)

        axs[i].set_title(name,fontsize=16)
        axs[i].set_xlabel('Num. threads', fontsize=xlabelfs)
        axs[i].set_xticks(x,labels=xlab,fontsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
    fig.suptitle(f'O4 improvement factor of each nth wrt baseline @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    
    axs[0].set_ylabel('Improvement Factor', fontsize=ylabelfs)
    axs[0].legend(labels,loc='best', fontsize= 14)
    # pp = PdfPages(os.path.join(hostname, f"Speedup_all_optim-{hostname}.pdf"))
    # pp.savefig(fig)
    # pp.close()
    #axs[i].show()
plot_improvement(resR)

In [ ]:
plot_improvement(resN)


# Time for best level

In [ ]:
def plot_bestlevel(results,step,maxNummode=False):
    #Configuration variables
    titlefs = 20
    ylabelfs = 18
    xlabelfs = 18
    xticksfs = 16
    yticksfs = 16
    legendfs = 14
    linew = 2
    markers = 8
    marks=['o-','x-','s-','v-','+-','.-','<-','>-','1-','2-','3','4','8','p','h','H','*','D','d','|','_']

    titles=['OWM Trav.','Tree Const.','Total']
    #fig = plt.figure()
    #labels=[x/10 for x in list(range(1,20,step))]
    if maxNummode:
        x=list(results["Alcoy"][3].keys())[1:-2:step]
    else:
        x=list(results["Alcoy"][3].keys())[::step]

    #define grid of plots
    fig, axs = plt.subplots(nrows=1, ncols=4,figsize=(15, 5), constrained_layout=True) #sharey=True
    for i,name in enumerate(['Alcoy','Arzua','BrionF','BrionU']):

        # plot the best configuration
        bestconf = get_best_comb_3d(results[name], 2)
        bestlevel = bestconf[0]
        bestminrad = bestconf[1]
        bestnth = bestconf[2]

        xticks=range(1,len(x)+1)
        for k,m in zip([1,0,2],marks):
            if(maxNummode):
               axs[i].plot(np.array(xticks), np.array([results[name][bestlevel][j][bestnth][k] for j in x]), m, linewidth=linew, markersize=markers)
            else:
               axs[i].plot(np.array(x), np.array([results[name][bestlevel][j][bestnth][k] for j in x]), m, linewidth=linew, markersize=markers)
        
        axs[i].set_title(f'{name} (lev={bestlevel})',fontsize=16)
        if(maxNummode):
            axs[i].set_xlabel('MaxNumber', fontsize=xlabelfs)
        else:
            axs[i].set_xlabel('MinRadius', fontsize=xlabelfs)
        if(maxNummode):    
            # axs[i].set_xticks(ticks=xticks[::3], labels=x[::3],fontsize=xticksfs)
            axs[i].set_xticks(ticks=xticks[::3], labels=x[::3])
        else:
            axs[i].set_xticks(x[::2])
        axs[i].tick_params(axis='x', labelsize=xticksfs)
        # axs[i].yticks(fontsize=yticksfs)
        axs[i].grid()
    if maxNummode:
        fig.suptitle(f'{maxth}-threads execution time (sec.) O4 MaxNumber @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    else:
        fig.suptitle(f'{maxth}-threads execution time (sec.) O4 MinRadius @ {hostname.upper()}',  fontweight='bold', fontsize=18)
    axs[0].set_ylabel('Time (sec.)', fontsize=ylabelfs)
    axs[0].legend(titles,loc='best', fontsize= 14)
    #axs[i].show()
    pp = None
    if(maxNummode):
        pp = PdfPages(f"{hostname}/MaxNum-{maxth}cores-time-{hostname}.pdf")
    else:
        pp = PdfPages(f"{hostname}/MinRad-{maxth}cores-time-{hostname}.pdf")
    pp.savefig(fig)
    pp.close()

plot_bestlevel(resR, 1)

plot_bestlevel(resN, 1, True)

In [ ]:
#For the best level (see next cell) find the min and max accuracy for all minRadius
def findbestlevel_config(res,phase):
    cloud_dict = {}
    # get the cloud list
    cloud_list = list(res.keys())
    for name in cloud_list:
        # print the best configuration
        bestconf = get_best_comb_3d(res[name], phase)
        bestlevel = bestconf[0]
        bestX = bestconf[1] # minRadius or maxNumber
        bestnth = bestconf[2]
        print(f'{name} {bestlevel} {bestX} {res[name][bestlevel][bestX][bestnth]}')
        cloud_dict[name] = {
            'level': bestlevel,
            'conf': bestX,
            'nth': bestnth,
            'tree': res[name][bestlevel][bestX][bestnth][0],
            'owm': res[name][bestlevel][bestX][bestnth][1],
            'total': res[name][bestlevel][bestX][bestnth][2],
            'accuracy': res[name][bestlevel][bestX][bestnth][3]
        }
    return cloud_dict

print("Best level and minRadius for each cloud, minimizing Tree Const. time")
print("Cloud, Level, MinRadius, [Tree Const.,OWM Trav., Total, Accuracy]")
findbestlevel_config(resR,0)
print("\nBest level and minRadius for each cloud, minimizing OWM traversal time")
print("Cloud, Level, MinRadius, [Tree Const., OWM Trav., Total, Accuracy]")
findbestlevel_config(resR,1)
print("\nBest level and minRadius for each cloud, minimizing Total time")
print("Cloud, Level, MinRadius, [Tree Const., OWM Trav., Total, Accuracy]")
bestTotalR = findbestlevel_config(resR,2)


In [ ]:
print("Best level and MaxNumber for each cloud, minimizing Tree Const. time")
print("Cloud, Level, MaxNumber, [Tree Const.,OWM Trav., Total, Accuracy]")
findbestlevel_config(resN,0)
print("\nBest level and MaxNumber for each cloud, minimizing OWM traversal time")
print("Cloud, Level, MaxNumber, [Tree Const., OWM Trav., Total, Accuracy]")
findbestlevel_config(resN,1)
print("\nBest level and MaxNumber for each cloud, minimizing Total time")
print("Cloud, Level, MaxNumber, [Tree Const., OWM Trav., Total, Accuracy]")
bestTotalN = findbestlevel_config(resN,2)

# Compute max and min error for different minradius and maxnumber values

In [ ]:
#For the best Level (see previous cell) find the min and max accuracy for all minRadius
print (" & \multicolumn{2}{|c|}{MinRadius} & \multicolumn{2}{|c|}{MaxNumber}\\\\ \hline")
print ("Cloud & Min accuracy & Max accuracy & Min accuracy & Max accuracy\\\\ \hline")
for cloud in resR:
    # get best configuration for minRadius
    bestconf = get_best_comb_3d(resR[cloud], 2)
    bestlevel = bestconf[0]
    bestminrad = bestconf[1]
    bestnth = bestconf[2]
    accuracyR=[resR[cloud][bestlevel][j][bestnth][3] for j in resR[cloud][bestlevel].keys()]
    minaccuracyR=min(accuracyR)
    maxaccuracyR=max(accuracyR)
    # get best configuration for maxNumber
    bestconf = get_best_comb_3d(resN[cloud], 2)
    bestlevel = bestconf[0]
    bestmaxnum = bestconf[1]
    bestnth = bestconf[2]
    accuracyN=[resN[cloud][bestlevel][j][bestnth][3] for j in resN[cloud][bestlevel].keys()]
    minaccuracyN=min(accuracyN)
    maxaccuracyN=max(accuracyN)
    print ("{} & {:0.2f}\% & {:0.2f}\% & {:0.2f}\% & {:0.2f}\%\\\\ \hline".format(i,minaccuracyR,maxaccuracyR,minaccuracyN,maxaccuracyN))

# Add final parallel (n-cores) times to All_Optimizations.out

In [ ]:
print(bestTotalR)
print(bestTotalN)

output = os.path.join(hostname, f'All_Optimizations-{hostname}.csv')

f = open(output, "a")

for cloud in bestTotalR:
    bestl=bestTotalR[cloud]['level'] # best level   
    bestmr=bestTotalR[cloud]['conf'] # best minRadius   
    treet=bestTotalR[cloud]['tree'] # tree time   
    owmt=bestTotalR[cloud]['owm'] #owm time   
    print("Opt4-MinRad;{};{:5f};{:5f};{};{} nth:{}".format(cloud,treet,owmt,bestl,bestmr, bestTotalR[cloud]['nth']))
    f.write("Opt4-MinRad;{};{:5f};{:5f};{};{}\n".format(cloud,treet,owmt,bestl,bestmr))

for cloud in bestTotalN:
    bestl=bestTotalN[cloud]['level'] # best level   
    bestmn=bestTotalN[cloud]['conf'] # best minRadius   
    treet=bestTotalN[cloud]['tree'] # tree time   
    owmt=bestTotalN[cloud]['owm'] #owm time   
    print("Opt4-MaxNum;{};{:5f};{:5f};{};{} nth:{}".format(cloud,treet,owmt,bestl,bestmn, bestTotalN[cloud]['nth']))
    f.write("Opt4-MaxNum;{};{:5f};{:5f};{};{}\n".format(cloud,treet,owmt,bestl,bestmn))

f.close()